In [0]:
# !pip install Faker names

In [0]:
import csv
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from faker import Faker

# Serviços disponíveis no hotel
servicos_hotel = {
    'Restaurante': {
        'itens': [
            {'descricao': 'Café da manhã', 'valor_min': 25.00, 'valor_max': 45.00},
            {'descricao': 'Almoço executivo', 'valor_min': 45.00, 'valor_max': 75.00},
            {'descricao': 'Jantar à la carte', 'valor_min': 80.00, 'valor_max': 150.00},
            {'descricao': 'Room service - jantar', 'valor_min': 60.00, 'valor_max': 120.00},
            {'descricao': 'Buffet completo', 'valor_min': 90.00, 'valor_max': 130.00}
        ]
    },
    'Bar': {
        'itens': [
            {'descricao': 'Drinks especiais', 'valor_min': 25.00, 'valor_max': 45.00},
            {'descricao': 'Vinho taça', 'valor_min': 30.00, 'valor_max': 60.00},
            {'descricao': 'Cervejas importadas', 'valor_min': 15.00, 'valor_max': 30.00},
            {'descricao': 'Coquetéis', 'valor_min': 35.00, 'valor_max': 55.00},
            {'descricao': 'Sucos naturais', 'valor_min': 12.00, 'valor_max': 20.00}
        ]
    },
    'Spa': {
        'itens': [
            {'descricao': 'Massagem relaxante', 'valor_min': 150.00, 'valor_max': 250.00},
            {'descricao': 'Tratamento facial', 'valor_min': 120.00, 'valor_max': 200.00},
            {'descricao': 'Day use spa', 'valor_min': 200.00, 'valor_max': 350.00},
            {'descricao': 'Massagem terapêutica', 'valor_min': 180.00, 'valor_max': 280.00},
            {'descricao': 'Pacote completo', 'valor_min': 300.00, 'valor_max': 500.00}
        ]
    },
    'Lavanderia': {
        'itens': [
            {'descricao': 'Lavagem roupa', 'valor_min': 40.00, 'valor_max': 80.00},
            {'descricao': 'Passagem roupa', 'valor_min': 25.00, 'valor_max': 50.00},
            {'descricao': 'Lavagem urgente', 'valor_min': 60.00, 'valor_max': 100.00},
            {'descricao': 'Lavagem a seco', 'valor_min': 35.00, 'valor_max': 70.00}
        ]
    },
    'Estacionamento': {
        'itens': [
            {'descricao': 'Diária estacionamento', 'valor_min': 30.00, 'valor_max': 50.00},
            {'descricao': 'Estacionamento por hora', 'valor_min': 10.00, 'valor_max': 15.00},
            {'descricao': 'Valet service', 'valor_min': 20.00, 'valor_max': 40.00}
        ]
    },
    'Business Center': {
        'itens': [
            {'descricao': 'Impressão página', 'valor_min': 2.00, 'valor_max': 5.00},
            {'descricao': 'Uso computador/hora', 'valor_min': 25.00, 'valor_max': 45.00},
            {'descricao': 'Reunião sala', 'valor_min': 100.00, 'valor_max': 200.00},
            {'descricao': 'Serviço secretária', 'valor_min': 50.00, 'valor_max': 100.00}
        ]
    }
}

def gerar_data_consumo():
    """Gera data de consumo aleatória entre 2020 e 2025"""
    # Definir período de 2020 a 2025
    data_inicio = datetime(2020, 1, 1)
    data_fim = datetime.today()
    
    # Calcular diferença total em dias
    diferenca_dias = (data_fim - data_inicio).days
    
    # Gerar data aleatória dentro do período
    dias_aleatorios = random.randint(0, diferenca_dias)
    data_consumo = data_inicio + timedelta(days=dias_aleatorios)
    
    return data_consumo

def gerar_consumo_servico():
    """Gera um consumo de serviço aleatório"""
    tipo_servico = random.choice(list(servicos_hotel.keys()))
    item = random.choice(servicos_hotel[tipo_servico]['itens'])
    
    valor = round(random.uniform(item['valor_min'], item['valor_max']), 2)
    
    # Probabilidade de pagamento: 80% pago, 20% não pago
    pago = random.choices(['Sim', 'Não'], weights=[80, 20])[0]
    
    return tipo_servico, item['descricao'], valor, pago

def gerar_consumos(total_consumos=5200):
    """Gera lista de consumos dos hóspedes"""
    consumos = []
    
    # Simular hóspedes ativos (com reservas recentes)
    hospedes_ativos = list(range(1, 201))  # 200 hóspedes possíveis
    quartos_ativos = list(range(101, 500))  # Quartos possíveis
    
    for i in range(total_consumos):
        consumo_id = 5001 + i
        
        # Um hóspede pode ter múltiplos consumos
        hospede_id = random.choice(hospedes_ativos)
        quarto_id = random.choice(quartos_ativos)
        
        data_consumo = gerar_data_consumo()
        tipo_servico, descricao, valor, pago = gerar_consumo_servico()
        
        consumo = (
            consumo_id,
            hospede_id,
            quarto_id,
            data_consumo,
            tipo_servico,
            descricao,
            valor,
            pago
        )
        
        consumos.append(consumo)
    
    return consumos

# Definir schema do DataFrame
schema = StructType([
    StructField("consumo_id", IntegerType(), True),
    StructField("hospede_id", IntegerType(), True),
    StructField("quarto_id", IntegerType(), True),
    StructField("data_consumo", DateType(), True),
    StructField("tipo_servico", StringType(), True),
    StructField("descricao", StringType(), True),
    StructField("valor", DoubleType(), True),
    StructField("pago", StringType(), True)
])

# Gerar os consumos
consumos = gerar_consumos(5200)

# Criar DataFrame Spark
df_consumos = spark.createDataFrame(consumos, schema)

# Mostrar algumas linhas para verificar as datas
print("Primeiras 15 linhas com datas variadas:")
df_consumos.show(15, truncate=False)

# Verificar distribuição por ano
print("\nDistribuição de consumos por ano:")
df_consumos.createOrReplaceTempView("consumos")
spark.sql("""
    SELECT YEAR(data_consumo) as ano, COUNT(*) as total_consumos
    FROM consumos 
    GROUP BY YEAR(data_consumo)
    ORDER BY ano
""").show()

# Salvar no catálogo do Databricks
catalog_path = "production.transient.consumos"

# Salvar como tabela Delta
df_consumos.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(catalog_path)

print(f"Arquivo salvo com sucesso em: {catalog_path}")
print(f"Total de consumos gravados: {df_consumos.count()}")

# OTIMIZAÇÃO - usando colunas mais relevantes para consultas
print("\nAplicando OPTIMIZE com ZORDER BY...")
spark.sql("""
OPTIMIZE production.transient.consumos
ZORDER BY (data_consumo, tipo_servico, hospede_id)
""")

print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from faker import Faker

# Carregar reservas do arquivo anterior (simulação)
# Na prática, você carregaria do CSV real
reservas_exemplo = [
    {'reserva_id': 1001, 'valor_total': '1500.00', 'status': 'Confirmada'},
    {'reserva_id': 1002, 'valor_total': '1200.00', 'status': 'Cancelada'},
    {'reserva_id': 1003, 'valor_total': '2000.00', 'status': 'Confirmada'},
    {'reserva_id': 1004, 'valor_total': '800.00', 'status': 'Pendente'},
    {'reserva_id': 1005, 'valor_total': '2700.00', 'status': 'Confirmada'}
]

# Formas de pagamento
formas_pagamento = ['Cartão crédito', 'Cartão débito', 'Pix', 'Dinheiro', 'Transferência', 'Boleto']

# Status de pagamento
status_pagamento = ['Pago', 'Não pago', 'Pendente', 'Reembolsado', 'Chargeback']

# Prefixos de notas fiscais por estado
prefixos_nf = {
    1: 'RJ',  # Rio de Janeiro
    2: 'SP',  # São Paulo
    3: 'RS',  # Rio Grande do Sul
    4: 'SC',  # Santa Catarina
    5: 'DF'   # Distrito Federal
}

def calcular_impostos(valor_total):
    """Calcula impostos (aproximadamente 10% do valor total)"""
    impostos = float(valor_total) * 0.10
    return round(impostos, 2)

def gerar_data_cobranca():
    """Gera data de cobrança aleatória entre 2020 e 2025"""
    # Definir período de 2020 a 2025
    data_inicio = datetime(2020, 1, 1)
    data_fim = datetime.today()
    
    # Calcular diferença total em dias
    diferenca_dias = (data_fim - data_inicio).days
    
    # Gerar data aleatória dentro do período
    dias_aleatorios = random.randint(0, diferenca_dias)
    data_cobranca = data_inicio + timedelta(days=dias_aleatorios)
    
    return data_cobranca

def gerar_status_pagamento(status_reserva):
    """Gera status de pagamento baseado no status da reserva"""
    if status_reserva == 'Cancelada':
        return random.choice(['Reembolsado', 'Não pago'])
    elif status_reserva == 'Confirmada':
        return random.choices(['Pago', 'Pendente'], weights=[80, 20])[0]
    else:  # Pendente ou outros
        return random.choice(['Pendente', 'Não pago'])

def gerar_nota_fiscal(reserva_id, hotel_id):
    """Gera número de nota fiscal"""
    prefixo = prefixos_nf.get(hotel_id, 'NF')
    numero_base = 12345 + reserva_id
    return f"{prefixo}{numero_base}"

def gerar_forma_pagamento(status_pagamento):
    """Gera forma de pagamento realista baseada no status"""
    if status_pagamento == 'Pago':
        return random.choices(['Cartão crédito', 'Pix', 'Cartão débito', 'Dinheiro'], 
                             weights=[50, 30, 15, 5])[0]
    elif status_pagamento == 'Pendente':
        return random.choice(['Boleto', 'Cartão crédito', 'Pix'])
    else:
        return random.choice(['Cartão crédito', 'Pix', 'Transferência'])

def gerar_faturas(total_faturas=5000):
    """Gera lista de faturas"""
    faturas = []
    
    # Assumindo que temos 200 reservas (de 1001 a 1200)
    for i in range(total_faturas):
        fatura_id = 9001 + i
        reserva_id = 1001 + (i % 5000)  # Cicla entre 1001 e 6001
        
        # Simular dados da reserva (na prática, carregaria do CSV)
        valor_total_reserva = random.uniform(500, 5000)
        status_reserva = random.choice(['Confirmada', 'Cancelada', 'Pendente'])
        hotel_id = random.randint(1, 5)
        
        valor_total = round(valor_total_reserva, 2)
        impostos = calcular_impostos(valor_total)
        status_pgto = gerar_status_pagamento(status_reserva)
        forma_pgto = gerar_forma_pagamento(status_pgto)
        data_cobranca = gerar_data_cobranca()
        nota_fiscal = gerar_nota_fiscal(reserva_id, hotel_id)
        
        fatura = (
            fatura_id,
            reserva_id,
            data_cobranca,
            valor_total,
            impostos,
            forma_pgto,
            status_pgto,
            nota_fiscal
        )
        
        faturas.append(fatura)
    
    return faturas

# Definir schema do DataFrame
schema = StructType([
    StructField("fatura_id", IntegerType(), True),
    StructField("reserva_id", IntegerType(), True),
    StructField("data_cobranca", DateType(), True),
    StructField("valor_total", DoubleType(), True),
    StructField("impostos", DoubleType(), True),
    StructField("forma_pagamento", StringType(), True),
    StructField("status_pagamento", StringType(), True),
    StructField("notas_fiscais", StringType(), True)
])

# Gerar as faturas
faturas = gerar_faturas(5000)

# Criar DataFrame Spark
df_faturas = spark.createDataFrame(faturas, schema)

# Mostrar algumas linhas para verificar as datas
print("Primeiras 15 linhas com datas variadas:")
df_faturas.show(15, truncate=False)

# Verificar distribuição por ano
print("\nDistribuição de faturas por ano:")
df_faturas.createOrReplaceTempView("faturas")
spark.sql("""
    SELECT YEAR(data_cobranca) as ano, 
           COUNT(*) as total_faturas,
           SUM(valor_total) as valor_total_ano,
           ROUND(AVG(valor_total), 2) as valor_medio
    FROM faturas 
    GROUP BY YEAR(data_cobranca)
    ORDER BY ano
""").show()

# Verificar status de pagamento por ano
print("\nStatus de pagamento por ano:")
spark.sql("""
    SELECT YEAR(data_cobranca) as ano,
           status_pagamento,
           COUNT(*) as total,
           ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY YEAR(data_cobranca)), 2) as percentual
    FROM faturas 
    GROUP BY YEAR(data_cobranca), status_pagamento
    ORDER BY ano, total DESC
""").show()

# Salvar no catálogo do Databricks
catalog_path = "production.transient.faturas"

# Salvar como tabela Delta
df_faturas.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(catalog_path)

print(f"Arquivo salvo com sucesso em: {catalog_path}")
print(f"Total de faturas gravadas: {df_faturas.count()}")

# OTIMIZAÇÃO - usando colunas mais relevantes para consultas
print("\nAplicando OPTIMIZE com ZORDER BY...")
spark.sql("""
OPTIMIZE production.transient.faturas
ZORDER BY (data_cobranca, forma_pagamento)
""")

print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from faker import Faker

# Configurar Faker para Brasil
fake_global = Faker('pt_BR')

# Lista de estados brasileiros e suas siglas
estados_brasileiros = [
    'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 
    'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 
    'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO'
]

# Tipos de cliente para hotelaria
tipos_cliente = ['Recorrente', 'Novo', 'VIP', 'Premium', 'Corporativo']

def gerar_cpf_valido():
    """Gera um CPF válido formatado"""
    def calcula_digito(cpf_parcial):
        soma = 0
        for i, digito in enumerate(cpf_parcial):
            soma += int(digito) * (len(cpf_parcial) + 1 - i)
        resto = soma % 11
        return '0' if resto < 2 else str(11 - resto)
    
    # Gerar 9 dígitos aleatórios
    cpf_parcial = [str(random.randint(0, 9)) for _ in range(9)]
    
    # Calcular primeiro dígito verificador
    cpf_parcial.append(calcula_digito(cpf_parcial))
    
    # Calcular segundo dígito verificador
    cpf_parcial.append(calcula_digito(cpf_parcial))
    
    # Formatar CPF
    cpf_formatado = f"{''.join(cpf_parcial[:3])}.{''.join(cpf_parcial[3:6])}.{''.join(cpf_parcial[6:9])}-{''.join(cpf_parcial[9:])}"
    
    return cpf_formatado

def gerar_data_nascimento():
    """Gera uma data de nascimento entre 18 e 80 anos atrás"""
    anos = random.randint(18, 80)
    dias = random.randint(0, 365)
    data_nasc = datetime.now() - timedelta(days=anos*365 + dias)
    return data_nasc

def gerar_telefone_brasileiro():
    """Gera telefone brasileiro no formato (XX) 9XXXX-XXXX"""
    ddd = random.choice(['11', '21', '31', '41', '51', '61', '71', '81', '91', 
                        '12', '22', '32', '42', '52', '62', '72', '82', '92',
                        '13', '23', '33', '43', '53', '63', '73', '83', '93',
                        '14', '24', '34', '44', '54', '64', '74', '84', '94',
                        '15', '25', '35', '45', '55', '65', '75', '85', '95',
                        '16', '26', '36', '46', '56', '66', '76', '86', '96',
                        '17', '27', '37', '47', '57', '67', '77', '87', '97',
                        '18', '28', '38', '48', '58', '68', '78', '88', '98',
                        '19', '29', '39', '49', '59', '69', '79', '89', '99'])
    
    numero = f"9{random.randint(1000, 9999)}-{random.randint(1000, 9999)}"
    return f"({ddd}) {numero}"

def gerar_email(nome):
    """Gera email baseado no nome"""
    nome_formatado = nome.lower().replace(' ', '.').replace('ã', 'a').replace('á', 'a').replace('â', 'a').replace('é', 'e').replace('ê', 'e').replace('í', 'i').replace('ó', 'o').replace('ô', 'o').replace('ú', 'u').replace('ç', 'c')
    dominios = ['gmail.com', 'hotmail.com', 'outlook.com', 'yahoo.com.br', 'bol.com.br', 'uol.com.br', 'ig.com.br']
    return f"{nome_formatado}@{random.choice(dominios)}"

def gerar_endereco_completo():
    """Gera endereço completo brasileiro"""
    estado = random.choice(estados_brasileiros)
    
    # Gerar cidade, bairro, rua e número
    cidade = fake_global.city()
    bairro = fake_global.bairro()
    rua = fake_global.street_name()
    numero = random.randint(1, 9999)
    complemento = random.choice(['', f'Apto {random.randint(1, 300)}', f'Casa {random.randint(1, 50)}', f'Bloco {random.randint(1, 10)}', 'Sala 101'])
    
    endereco = {
        'estado': estado,
        'cidade': cidade,
        'bairro': bairro,
        'rua': rua,
        'numero': numero,
        'complemento': complemento,
        'cep': fake_global.postcode()
    }
    
    return endereco

def gerar_nome_brasileiro(sexo):
    """Gera nome brasileiro"""
    if sexo == 'M':
        return fake_global.name_male()
    else:
        return fake_global.name_female()

def gerar_hospedes(total_hospedes=1200):
    """Gera lista de hóspedes brasileiros"""
    hospedes = []
    
    for i in range(total_hospedes):
        sexo = random.choice(['M', 'F'])
        
        # Gerar dados pessoais
        nome = gerar_nome_brasileiro(sexo)
        cpf = gerar_cpf_valido()
        data_nascimento = gerar_data_nascimento()
        
        # Gerar endereço completo
        endereco = gerar_endereco_completo()
        
        hospede = (
            i + 1,  # hospede_id
            nome,
            cpf,
            data_nascimento,
            sexo,
            'Brasil',  # nacionalidade fixa
            random.choice(tipos_cliente),
            gerar_email(nome),
            gerar_telefone_brasileiro(),
            endereco['estado'],
            endereco['cidade'],
            endereco['bairro'],
            endereco['rua'],
            endereco['numero'],
            endereco['complemento'],
            endereco['cep']
        )
        
        hospedes.append(hospede)
    
    return hospedes

# Definir schema do DataFrame atualizado
schema = StructType([
    StructField("hospede_id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("cpf", StringType(), True),
    StructField("data_nascimento", DateType(), True),
    StructField("sexo", StringType(), True),
    StructField("nacionalidade", StringType(), True),
    StructField("tipo_cliente", StringType(), True),
    StructField("email", StringType(), True),
    StructField("telefone", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True),
    StructField("bairro", StringType(), True),
    StructField("rua", StringType(), True),
    StructField("numero", IntegerType(), True),
    StructField("complemento", StringType(), True),
    StructField("cep", StringType(), True)
])

# Gerar os hóspedes
hospedes = gerar_hospedes(1200)

# Criar DataFrame Spark
df_hospedes = spark.createDataFrame(hospedes, schema)

# Salvar no catálogo do Databricks
catalog_path = "production.transient.hospedes"

# Salvar como tabela Delta
df_hospedes.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(catalog_path)

print(f"Arquivo salvo com sucesso em: {catalog_path}")
print(f"Total de hóspedes gravados: {df_hospedes.count()}")

# OTIMIZAÇÃO - usando colunas mais relevantes para consultas
print("\nAplicando OPTIMIZE com ZORDER BY...")
spark.sql("""
OPTIMIZE production.transient.hospedes
ZORDER BY (estado, cidade, data_nascimento)
""")

print("Otimização concluída com sucesso!")

In [0]:
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from faker import Faker

# Dados dos hotéis
hoteis = [
    {'hotel_id': 1, 'nome': 'Hotel Praia Dourada', 'cidade': 'Rio de Janeiro', 'andares': 8, 'quartos_por_andar': 20},
    {'hotel_id': 2, 'nome': 'Hotel Business Center', 'cidade': 'São Paulo', 'andares': 12, 'quartos_por_andar': 15},
    {'hotel_id': 3, 'nome': 'Mountain Resort', 'cidade': 'Gramado', 'andares': 4, 'quartos_por_andar': 25},
    {'hotel_id': 4, 'nome': 'Beach Paradise', 'cidade': 'Florianópolis', 'andares': 6, 'quartos_por_andar': 18},
    {'hotel_id': 5, 'nome': 'Capital Suites', 'cidade': 'Brasília', 'andares': 10, 'quartos_por_andar': 12}
]

# Tipos de quarto com suas características
tipos_quarto = {
    'Standard': {
        'capacidade': [1, 2],
        'faixa_preco': [250.00, 400.00],
        'descricao': 'Quarto padrão com cama de casal ou duas camas de solteiro'
    },
    'Luxo': {
        'capacidade': [2, 3],
        'faixa_preco': [450.00, 650.00],
        'descricao': 'Quarto espaçoso com amenities premium'
    },
    'Suíte': {
        'capacidade': [2, 4],
        'faixa_preco': [600.00, 900.00],
        'descricao': 'Suíte com área de estar separada'
    },
    'Suíte Master': {
        'capacidade': [3, 4],
        'faixa_preco': [800.00, 1200.00],
        'descricao': 'Suíte master com hidromassagem'
    },
    'Suíte Presidencial': {
        'capacidade': [4, 6],
        'faixa_preco': [1500.00, 3000.00],
        'descricao': 'Suíte presidencial com todos os luxos'
    }
}

# Status possíveis para os quartos
status_quarto = ['Disponível', 'Ocupado', 'Manutenção', 'Limpeza', 'Reservado']

def gerar_quartos():
    """Gera a lista de quartos para todos os hotéis"""
    quartos = []
    
    for hotel in hoteis:
        hotel_id = hotel['hotel_id']
        andares = hotel['andares']
        quartos_por_andar = hotel['quartos_por_andar']
        
        # Distribuição dos tipos de quarto por andar
        distribuicao_tipos = {
            'Standard': 0.50,  # 50% dos quartos
            'Luxo': 0.25,      # 25% dos quartos
            'Suíte': 0.15,     # 15% dos quartos
            'Suíte Master': 0.07,  # 7% dos quartos
            'Suíte Presidencial': 0.03  # 3% dos quartos
        }
        
        quarto_numero = 1
        
        for andar in range(1, andares + 1):
            quartos_no_andar = []
            
            # Determinar quantos quartos de cada tipo neste andar
            total_quartos_andar = quartos_por_andar
            quartos_standard = int(total_quartos_andar * distribuicao_tipos['Standard'])
            quartos_luxo = int(total_quartos_andar * distribuicao_tipos['Luxo'])
            quartos_suite = int(total_quartos_andar * distribuicao_tipos['Suíte'])
            quartos_master = int(total_quartos_andar * distribuicao_tipos['Suíte Master'])
            quartos_presidencial = int(total_quartos_andar * distribuicao_tipos['Suíte Presidencial'])
            
            # Ajustar para totalizar o número correto
            total_atual = quartos_standard + quartos_luxo + quartos_suite + quartos_master + quartos_presidencial
            if total_atual < total_quartos_andar:
                quartos_standard += total_quartos_andar - total_atual
            
            # Adicionar quartos Standard
            for _ in range(quartos_standard):
                quartos_no_andar.append('Standard')
            
            # Adicionar quartos Luxo (andares mais altos têm mais quartos luxo)
            if andar >= 3:
                for _ in range(quartos_luxo):
                    quartos_no_andar.append('Luxo')
            
            # Adicionar Suítes (andares mais altos)
            if andar >= 5:
                for _ in range(quartos_suite):
                    quartos_no_andar.append('Suíte')
            
            # Adicionar Suítes Master (andares altos)
            if andar >= 7:
                for _ in range(quartos_master):
                    quartos_no_andar.append('Suíte Master')
            
            # Adicionar Suítes Presidenciais (últimos andares)
            if andar >= andares - 1:
                for _ in range(quartos_presidencial):
                    quartos_no_andar.append('Suíte Presidencial')
            
            # Embaralhar a distribuição no andar
            random.shuffle(quartos_no_andar)
            
            # Criar quartos para este andar
            for tipo in quartos_no_andar:
                quarto_id = andar * 100 + quarto_numero
                quarto_numero += 1
                
                # Capacidade baseada no tipo
                capacidade_min, capacidade_max = tipos_quarto[tipo]['capacidade']
                capacidade = random.randint(capacidade_min, capacidade_max)
                
                # Preço base baseado no tipo e andar (andares mais altos são mais caros)
                preco_min, preco_max = tipos_quarto[tipo]['faixa_preco']
                preco_base = random.uniform(preco_min, preco_max)
                preco_base += (andar - 1) * 10  # Acréscimo por andar
                
                # Status do quarto (maior probabilidade de disponível)
                pesos_status = [70, 15, 8, 5, 2]  # Probabilidades relativas
                status = random.choices(status_quarto, weights=pesos_status)[0]
                
                quarto = (
                    quarto_id,
                    hotel_id,
                    tipo,
                    capacidade,
                    andar,
                    status,
                    round(preco_base, 2)
                )
                
                quartos.append(quarto)
    
    return quartos

# Definir schema do DataFrame
schema = StructType([
    StructField("quarto_id", IntegerType(), True),
    StructField("hotel_id", IntegerType(), True),
    StructField("tipo_quarto", StringType(), True),
    StructField("capacidade", IntegerType(), True),
    StructField("andar", IntegerType(), True),
    StructField("status_quarto", StringType(), True),
    StructField("preco_base", DoubleType(), True)
])

# Gerar os quartos
quartos = gerar_quartos()

# Criar DataFrame Spark
df_quartos = spark.createDataFrame(quartos, schema)

# Salvar no catálogo do Databricks
catalog_path = "production.transient.quartos"

# Salvar como tabela Delta
df_quartos.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(catalog_path)

print(f"Arquivo salvo com sucesso em: {catalog_path}")
print(f"Total de quartos gravados: {df_quartos.count()}")

# OTIMIZAÇÃO - usando colunas mais relevantes para consultas
print("\nAplicando OPTIMIZE com ZORDER BY...")
spark.sql("""
OPTIMIZE production.transient.quartos
ZORDER BY (tipo_quarto, capacidade, andar)
""")

print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from faker import Faker

# OTAs (Online Travel Agencies) com suas comissões típicas
otas = [
    {'nome': 'Booking', 'comissao_min': 12.0, 'comissao_max': 18.0, 'prefixo': 'BK'},
    {'nome': 'Expedia', 'comissao_min': 15.0, 'comissao_max': 20.0, 'prefixo': 'EX'},
    {'nome': 'Airbnb', 'comissao_min': 10.0, 'comissao_max': 15.0, 'prefixo': 'AB'},
    {'nome': 'Decolar', 'comissao_min': 12.0, 'comissao_max': 16.0, 'prefixo': 'DE'},
    {'nome': 'CVC', 'comissao_min': 8.0, 'comissao_max': 12.0, 'prefixo': 'CV'},
    {'nome': 'Tripadvisor', 'comissao_min': 13.0, 'comissao_max': 17.0, 'prefixo': 'TA'},
    {'nome': 'Hotels.com', 'comissao_min': 14.0, 'comissao_max': 19.0, 'prefixo': 'HO'},
    {'nome': 'Trivago', 'comissao_min': 11.0, 'comissao_max': 15.0, 'prefixo': 'TR'}
]

# Canais diretos (sem comissão)
canais_diretos = ['Site', 'Telefone', 'WhatsApp', 'Presencial', 'Email']

def calcular_comissao_por_tipo_quarto(tipo_quarto):
    """Calcula comissão ajustada por tipo de quarto"""
    ajustes = {
        'Standard': 0.0,      # Comissão padrão
        'Luxo': 1.0,          # +1% para quartos de luxo
        'Suíte': 2.0,         # +2% para suítes
        'Suíte Master': 3.0,  # +3% para suítes master
        'Suíte Presidencial': 4.0  # +4% para suítes presidenciais
    }
    return ajustes.get(tipo_quarto, 0.0)

def gerar_data_reserva():
    """Gera data de reserva aleatória entre 2020 e 2025"""
    # Definir período de 2020 a 2025
    data_inicio = datetime(2020, 1, 1)
    data_fim = datetime(2025, 12, 31)
    
    # Calcular diferença total em dias
    diferenca_dias = (data_fim - data_inicio).days
    
    # Gerar data aleatória dentro do período
    dias_aleatorios = random.randint(0, diferenca_dias)
    data_reserva = data_inicio + timedelta(days=dias_aleatorios)
    
    return data_reserva

def gerar_reserva_ota(reserva_id):
    """Gera uma reserva OTA para uma reserva existente"""
    # 70% das reservas vêm de OTAs, 30% de canais diretos
    if random.random() > 0.7:
        return None  # Reserva direta, sem OTA
    
    ota = random.choice(otas)
    comissao_percent = round(random.uniform(ota['comissao_min'], ota['comissao_max']), 1)
    
    # Gerar ID da OTA (prefixo + número sequencial)
    ota_id_number = 8001 + reserva_id - 1001  # Mantém relação com reserva_id
    reserva_ota_id = f"{ota['prefixo']}{ota_id_number}"
    
    data_reserva = gerar_data_reserva()
    
    return (
        reserva_ota_id,
        reserva_id,
        ota['nome'],
        comissao_percent,
        data_reserva
    )

def gerar_reservas_otas(total_reservas=2200):
    """Gera lista de reservas OTA"""
    reservas_ota = []
    
    # Simular reservas existentes (de 1001 a 1200)
    for reserva_id in range(1001, 1001 + total_reservas):
        # Simular tipo de quarto para ajuste de comissão
        tipo_quarto = random.choice(['Standard', 'Luxo', 'Suíte', 'Suíte Master', 'Suíte Presidencial'])
        
        reserva_ota = gerar_reserva_ota(reserva_id)
        if reserva_ota:
            # Ajustar comissão baseada no tipo de quarto
            ajuste = calcular_comissao_por_tipo_quarto(tipo_quarto)
            comissao_ajustada = round(reserva_ota[3] + ajuste, 1)
            
            # Criar nova tupla com comissão ajustada
            reserva_ota_ajustada = (
                reserva_ota[0],  # reserva_ota_id
                reserva_ota[1],  # reserva_id
                reserva_ota[2],  # canal
                comissao_ajustada,  # comissao_percent
                reserva_ota[4]   # data_reserva
            )
            reservas_ota.append(reserva_ota_ajustada)
    
    return reservas_ota

# Definir schema do DataFrame
schema = StructType([
    StructField("reserva_ota_id", StringType(), True),
    StructField("reserva_id", IntegerType(), True),
    StructField("canal", StringType(), True),
    StructField("comissao_percent", DoubleType(), True),
    StructField("data_reserva", DateType(), True)
])

# Gerar as reservas OTA
reservas_ota = gerar_reservas_otas(2000)

# Criar DataFrame Spark
df_reservas_ota = spark.createDataFrame(reservas_ota, schema)

# Mostrar algumas linhas para verificar as datas
print("Primeiras 15 linhas com datas variadas:")
df_reservas_ota.show(15, truncate=False)

# Verificar distribuição por ano
print("\nDistribuição de reservas OTA por ano:")
df_reservas_ota.createOrReplaceTempView("reservas_ota")
spark.sql("""
    SELECT YEAR(data_reserva) as ano, 
           COUNT(*) as total_reservas,
           ROUND(AVG(comissao_percent), 2) as comissao_media
    FROM reservas_ota 
    GROUP BY YEAR(data_reserva)
    ORDER BY ano
""").show()

# Verificar distribuição por canal
print("\nDistribuição por canal:")
spark.sql("""
    SELECT canal, 
           COUNT(*) as total_reservas,
           ROUND(AVG(comissao_percent), 2) as comissao_media
    FROM reservas_ota 
    GROUP BY canal
    ORDER BY total_reservas DESC
""").show()

# Salvar no catálogo do Databricks
catalog_path = "production.transient.reservas_ota"

# Salvar como tabela Delta
df_reservas_ota.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(catalog_path)

print(f"Arquivo salvo com sucesso em: {catalog_path}")
print(f"Total de reservas OTA gravadas: {df_reservas_ota.count()}")

# OTIMIZAÇÃO - usando colunas mais relevantes para consultas
print("\nAplicando OPTIMIZE com ZORDER BY...")
spark.sql("""
OPTIMIZE production.transient.reservas_ota
ZORDER BY (data_reserva, comissao_percent)
""")

print("Otimização concluída com sucesso!")

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from faker import Faker

# Configurar Faker
fake = Faker('pt_BR')

# Dados base para as reservas
hoteis = [
    {'hotel_id': 1, 'nome': 'Hotel Praia Dourada', 'cidade': 'Rio de Janeiro'},
    {'hotel_id': 2, 'nome': 'Hotel Business Center', 'cidade': 'São Paulo'},
    {'hotel_id': 3, 'nome': 'Mountain Resort', 'cidade': 'Gramado'},
    {'hotel_id': 4, 'nome': 'Beach Paradise', 'cidade': 'Florianópolis'},
    {'hotel_id': 5, 'nome': 'Capital Suites', 'cidade': 'Brasília'}
]

# Quartos por hotel (quarto_id: [hotel_id, tipo, capacidade, diaria_base])
quartos = {
    101: [1, 'Standard', 2, 300.00],
    102: [1, 'Standard', 2, 300.00],
    201: [1, 'Luxo', 2, 500.00],
    202: [1, 'Luxo', 2, 500.00],
    301: [1, 'Suíte', 4, 800.00],
    103: [2, 'Standard', 2, 350.00],
    104: [2, 'Standard', 2, 350.00],
    203: [2, 'Luxo', 2, 600.00],
    303: [2, 'Suíte', 4, 900.00],
    105: [3, 'Standard', 2, 400.00],
    205: [3, 'Luxo', 2, 700.00],
    305: [3, 'Suíte', 4, 1100.00],
    106: [4, 'Standard', 2, 250.00],
    206: [4, 'Luxo', 2, 450.00],
    107: [5, 'Standard', 2, 320.00],
    207: [5, 'Luxo', 2, 550.00]
}

canais_reserva = ['Site', 'Booking', 'Expedia', 'Agência', 'Telefone', 'WhatsApp', 'Presencial']
status_reserva = ['Confirmada', 'Cancelada', 'Pendente', 'Check-in', 'Check-out', 'No Show']
observacoes_comuns = ['-', 'Cliente VIP', 'Aniversariante', 'Honeymoon', 'Cancelamento por covid', 
                     'Reagendamento', 'Pagamento pendente', 'Early check-in solicitado',
                     'Late check-out solicitado', 'Alergia a frutos do mar', 'Cama extra necessária']

def gerar_data_aleatoria_2020_2025():
    """Gera data aleatória entre 2020 e 2025"""
    data_inicio = datetime(2020, 1, 1)
    data_fim = datetime(2025, 12, 31)
    
    diferenca_dias = (data_fim - data_inicio).days
    dias_aleatorios = random.randint(0, diferenca_dias)
    
    return data_inicio + timedelta(days=dias_aleatorios)

def gerar_data_reserva():
    """Gera data de reserva entre 2020 e 2025"""
    return gerar_data_aleatoria_2020_2025()

def gerar_datas_checkin_checkout(data_reserva):
    """Gera datas de check-in e check-out baseadas na data de reserva"""
    # Check-in entre 1 e 90 dias após a reserva
    dias_ate_checkin = random.randint(1, 90)
    # Estadia de 1 a 14 noites
    noites = random.randint(1, 14)
    
    data_checkin = data_reserva + timedelta(days=dias_ate_checkin)
    data_checkout = data_checkin + timedelta(days=noites)
    
    # Garantir que as datas não ultrapassem 2025
    data_maxima = datetime(2025, 12, 31)
    if data_checkout > data_maxima:
        data_checkout = data_maxima
        # Recalcular noites baseado na data máxima
        noites = (data_checkout - data_checkin).days
        if noites <= 0:
            # Se não há noites possíveis, ajustar check-in
            data_checkin = data_checkout - timedelta(days=1)
            noites = 1
    
    return data_checkin, data_checkout, noites

def calcular_valor_total(quarto_id, noites, status):
    """Calcula valor total com possibilidade de desconto"""
    diaria_base = quartos[quarto_id][3]
    valor_base = diaria_base * noites
    
    # Aplicar desconto baseado no canal de reserva e status
    if status == 'Cancelada':
        desconto = valor_base  # Reembolso total para canceladas
    else:
        # Descontos variados baseados no tipo de quarto
        tipo_quarto = quartos[quarto_id][1]
        if tipo_quarto == 'Standard':
            desconto = random.choice([0, 20, 50, 80])
        elif tipo_quarto == 'Luxo':
            desconto = random.choice([0, 50, 100, 150])
        else:  # Suítes
            desconto = random.choice([0, 100, 200, 300])
    
    valor_total = max(0, valor_base - desconto)
    return round(valor_total, 2), round(desconto, 2)

def gerar_reservas(total_reservas=5000):
    """Gera lista de reservas"""
    reservas = []
    
    for i in range(total_reservas):
        reserva_id = 1000 + i + 1
        hospede_id = random.randint(1, 200)  # Assumindo 200 hóspedes do arquivo anterior
        quarto_id = random.choice(list(quartos.keys()))
        hotel_id = quartos[quarto_id][0]
        
        data_reserva = gerar_data_reserva()
        data_checkin, data_checkout, noites = gerar_datas_checkin_checkout(data_reserva)
        
        canal = random.choice(canais_reserva)
        status = random.choice(status_reserva)
        
        valor_total, desconto = calcular_valor_total(quarto_id, noites, status)
        observacao = random.choice(observacoes_comuns)
        
        reserva = (
            reserva_id,
            hospede_id,
            quarto_id,
            hotel_id,
            data_reserva,
            data_checkin,
            data_checkout,
            canal,
            status,
            valor_total,
            desconto,
            observacao
        )
        
        reservas.append(reserva)
    
    return reservas

# Definir schema do DataFrame
schema = StructType([
    StructField("reserva_id", IntegerType(), True),
    StructField("hospede_id", IntegerType(), True),
    StructField("quarto_id", IntegerType(), True),
    StructField("hotel_id", IntegerType(), True),
    StructField("data_reserva", DateType(), True),
    StructField("data_checkin", DateType(), True),
    StructField("data_checkout", DateType(), True),
    StructField("canal_reserva", StringType(), True),
    StructField("status", StringType(), True),
    StructField("valor_total", DoubleType(), True),
    StructField("desconto", DoubleType(), True),
    StructField("observacoes", StringType(), True)
])

# Gerar as reservas
reservas = gerar_reservas(5000)

# Criar DataFrame Spark
df_reservas = spark.createDataFrame(reservas, schema)

# Mostrar algumas linhas para verificar as datas
print("Primeiras 15 linhas com datas variadas:")
df_reservas.show(15, truncate=False)

# Verificar distribuição por ano
print("\nDistribuição de reservas por ano:")
df_reservas.createOrReplaceTempView("reservas")
spark.sql("""
    SELECT YEAR(data_reserva) as ano, 
           COUNT(*) as total_reservas,
           SUM(CASE WHEN status = 'Cancelada' THEN 1 ELSE 0 END) as canceladas,
           ROUND(AVG(valor_total), 2) as valor_medio,
           ROUND(SUM(valor_total), 2) as valor_total_ano
    FROM reservas 
    GROUP BY YEAR(data_reserva)
    ORDER BY ano
""").show()

# Verificar status das reservas
print("\nStatus das reservas:")
spark.sql("""
    SELECT status, 
           COUNT(*) as total,
           ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM reservas), 2) as percentual
    FROM reservas 
    GROUP BY status
    ORDER BY total DESC
""").show()

# Verificar distribuição por canal
print("\nDistribuição por canal de reserva:")
spark.sql("""
    SELECT canal_reserva, 
           COUNT(*) as total_reservas,
           ROUND(AVG(valor_total), 2) as valor_medio
    FROM reservas 
    GROUP BY canal_reserva
    ORDER BY total_reservas DESC
""").show()

# Salvar no catálogo do Databricks
catalog_path = "production.transient.reservas"

# Salvar como tabela Delta
df_reservas.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(catalog_path)

print(f"Arquivo salvo com sucesso em: {catalog_path}")
print(f"Total de reservas gravadas: {df_reservas.count()}")

# OTIMIZAÇÃO - usando colunas mais relevantes para consultas
print("\nAplicando OPTIMIZE with ZORDER BY...")
spark.sql("""
OPTIMIZE production.transient.reservas
ZORDER BY (data_reserva, data_checkin, data_checkout)
""")

print("Otimização concluída com sucesso!")